<a href="https://colab.research.google.com/github/Promila-uwc/DeepSpeech-examples/blob/master/Cosine_Similarity_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **Cosine Similarity Operation**

In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import spacy
nlp = spacy.load("en_core_web_sm")

def text_processing(sentence):
    """
    Lemmatize, lowercase, remove numbers and stop words
    
    Args:
      sentence: The sentence we want to process.
    
    Returns:
      A list of processed words
    """
    sentence = [token.lemma_.lower()
                for token in nlp(sentence) 
                if token.is_alpha and not token.is_stop]
    
    return sentence


def cos_sim(sentence1_emb, sentence2_emb):
    """
    Cosine similarity between two columns of sentence embeddings
    
    Args:
      sentence1_emb: sentence1 embedding column
      sentence2_emb: sentence2 embedding column
    
    Returns:
      The row-wise cosine similarity between the two columns.
      For instance is sentence1_emb=[a,b,c] and sentence2_emb=[x,y,z]
      Then the result is [cosine_similarity(a,x), cosine_similarity(b,y), cosine_similarity(c,z)]
    """
    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

In [ ]:
import pandas as pd
  
# Creating Empty DataFrame and Storing it in variable df
df = pd.DataFrame()

In [ ]:
# Imports
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# Load the English STSB dataset
stsb_dataset = load_dataset('stsb_multi_mt', 'en')
stsb_train = pd.DataFrame(stsb_dataset['train'])
stsb_test = pd.DataFrame(stsb_dataset['test'])

# Check loaded data
print(stsb_train.shape, stsb_test.shape)
stsb_train.head()
df["sentence1"] = stsb_test["sentence1"]
df["sentence2"] = stsb_test["sentence2"]

  0%|          | 0/3 [00:00<?, ?it/s]

(5749, 3) (1379, 3)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# model = TfidfVectorizer(lowercase=True, stop_words='english')

# # Train the model
# X_train = pd.concat([stsb_train['sentence1'], stsb_train['sentence2']]).unique()
# model.fit(X_train)
model = SentenceTransformer('bert-base-nli-mean-tokens')
# Generate Embeddings on Test
sentence1_emb = model.encode(stsb_test['sentence1'])
sentence2_emb = model.encode(stsb_test['sentence2'])

# Cosine Similarity
stsb_test['BERT_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)

In [ ]:
print(stsb_test['BERT_cosine_score'])

0       0.914903
1       0.973847
2       0.958104
3       0.980779
4       0.466283
          ...   
1374    0.365521
1375    0.481370
1376    0.463050
1377    0.587686
1378    0.502197
Name: BERT_cosine_score, Length: 1379, dtype: float32


In [ ]:
df["BERT_cosine_score"]=stsb_test['BERT_cosine_score']

In [ ]:
df.head()

,sentence1,sentence2,BERT_cosine_score
0,A girl is styling her hair.,A girl is brushing her hair.,0.914903
1,A group of men play soccer on the beach.,A group of boys are playing soccer on the beach.,0.973847
2,One woman is measuring another woman's ankle.,A woman measures another woman's ankle.,0.958104
3,A man is cutting up a cucumber.,A man is slicing a cucumber.,0.980779
4,A man is playing a harp.,A man is playing a keyboard.,0.466283


In [ ]:
# saving the dataframe
df.to_csv('cosine_similarity_bert.csv')